In [1]:
#importing the required libraries
import os
import cv2
import glob
import zipfile
import numpy as np
from tqdm import tqdm
from urllib import request

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#you need to replace this by the location of the dataset on your drive
# datasetloc="/content/drive/MyDrive/Hemangini Projects/dataset"
#! wget -q https://storage.googleapis.com/drive-bulk-export-anonymous/20220428T122708Z/4133399871716478688/aa76fc5c-9354-4b5f-ae56-aca2048e1129/1/acf3a9e4-29f2-415b-86b6-f7ba375e19ee?authuser -O dataset.zip
# drive.mount('/content/drive')
# datasetloc="/content/drive/MyDrive/Colab Notebooks/dataset"

# ! wget -q https://tinyurl.com/r5ut6tjx -O dataset.zip
# ! unzip -oqq dataset
# datasetloc="./dataset"

if not os.path.exists("./dataset.zip"):
    response = request.urlretrieve("https://tinyurl.com/5acvv2en", "dataset.zip")
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    datasetloc="./dataset"

In [ ]:
fakefiles=glob.glob(datasetloc+"/training_fake/*.jpg")

In [ ]:
#reading the fake images and labelling them as 1
#tqdm helps to keep track of progress
data=[]
y=[]
for f in tqdm(fakefiles):
  twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  
  data.append(img)
  y.append(1)

100%|██████████| 960/960 [00:12<00:00, 75.77it/s]


In [ ]:
realfiles=glob.glob(datasetloc+"/training_real/*.jpg")

In [ ]:
#reading the real images and labelling them as 0
for f in tqdm(realfiles):
  twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  data.append(img)
  y.append(0)
  

100%|██████████| 1081/1081 [00:14<00:00, 74.04it/s]


In [ ]:
#convering lists into arrays
data = np.asarray(data)
y = np.asarray(y)

In [ ]:
#then we do the test train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [ ]:
#importing the libraries required for our cnn model
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (128,128,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
#fitting our model on the dataset
model.fit(X_train,y_train,epochs=10,batch_size=40)

Epoch 1/10
41/41 [==============================] - 57s 1s/step - loss: 37.7185 - accuracy: 0.5147
Epoch 2/10
41/41 [==============================] - 56s 1s/step - loss: 0.7062 - accuracy: 0.5055
Epoch 3/10
41/41 [==============================] - 56s 1s/step - loss: 0.6927 - accuracy: 0.5355
Epoch 4/10
41/41 [==============================] - 56s 1s/step - loss: 0.6902 - accuracy: 0.5288
Epoch 5/10
41/41 [==============================] - 55s 1s/step - loss: 0.6939 - accuracy: 0.5208
Epoch 6/10
41/41 [==============================] - 55s 1s/step - loss: 0.6923 - accuracy: 0.5374
Epoch 7/10
41/41 [==============================] - 55s 1s/step - loss: 0.6824 - accuracy: 0.5435
Epoch 8/10
41/41 [==============================] - 56s 1s/step - loss: 0.6839 - accuracy: 0.5545
Epoch 9/10
41/41 [==============================] - 59s 1s/step - loss: 0.6848 - accuracy: 0.5527
Epoch 10/10
41/41 [==============================] - 56s 1s/step - loss: 0.6759 - accuracy: 0.5570


In [ ]:
ypredtrain=model.predict(X_train)
ypredtest=model.predict(X_test)

In [ ]:
yptrain=[]
yptest=[]
for y in ypredtrain:
  if(y>=0.5):
    yptrain.append(1)
  else:
    yptrain.append(0)
for y in ypredtest:
  if(y>=0.5):
    yptest.append(1)
  else:
    yptest.append(0)


In [ ]:
from sklearn.metrics import accuracy_score
print("the accuracy score for training set is",accuracy_score(y_train, yptrain))
print("the accuracy score for test set is",accuracy_score(y_test, yptest))

the accuracy score for training set is 0.5594362745098039
the accuracy score for test set is 0.5232273838630807
